In [58]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [59]:
os.listdir()
#os.chdir(r'C:\Users\decmg\OneDrive\Documentos\Material Disciplinas\MO444')

['.gitignore', 'data', 'eda_proj_final.ipynb', 'Untitled-1.ipynb']

In [61]:
# Dados disponíveis em:
# - Numpy: https://drive.google.com/drive/u/0/folders/1tz5eVPwGx4kTYKwbZAQLLbVmFQxZva0Q
# - raw: https://openneuro.org/datasets/ds004504/versions/1.0.7

## Manipulando os dados para chegar ao formato desejado

In [ ]:
def load_train_val_test_list(label_path, a=0.6, b=0.8):

    data_list = np.load(label_path)

    cn_list = list(data_list[np.where(data_list[:, 0] == 0)][:, 1])  # healthy IDs
    print(cn_list)
    #ftd_list = list(
    #    data_list[np.where(data_list[:, 0] == 1)][:, 1]
    #)  # Frontotemporal Dementia IDs
    #print(ftd_list)
    ad_list = list(
        data_list[np.where(data_list[:, 0] == 2)][:, 1]
    )  # Alzheimer's disease IDs
    print(ad_list)
    train_ids = (
        cn_list[: int(a * len(cn_list))]
        #+ ftd_list[: int(a * len(ftd_list))]
        + ad_list[: int(a * len(ad_list))]
    )
    val_ids = (
        cn_list[int(a * len(cn_list)) : int(b * len(cn_list))]
        #+ ftd_list[int(a * len(ftd_list)) : int(b * len(ftd_list))]
        + ad_list[int(a * len(ad_list)) : int(b * len(ad_list))]
    )
    test_ids = (
        cn_list[int(b * len(cn_list)) :]
        #+ ftd_list[int(b * len(ftd_list)) :]
        + ad_list[int(b * len(ad_list)) :]
    )
    return train_ids, val_ids, test_ids

In [ ]:
label_path = 'data/ADFD/Label/label.npy'
train_ids, val_ids, test_ids = load_train_val_test_list(label_path, a=0.6, b=0.8)
print(f'--> {len(train_ids)} : {train_ids}')
print(f'--> {len(val_ids)} : {val_ids}')
print(f'--> {len(test_ids)} : {test_ids}')

[37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
--> 38 : [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
--> 13 : [54, 55, 56, 57, 58, 59, 22, 23, 24, 25, 26, 27, 28]
--> 14 : [60, 61, 62, 63, 64, 65, 29, 30, 31, 32, 33, 34, 35, 36]


In [ ]:
import numpy as np
from sklearn.utils import shuffle

# Load labels
labels = np.load('data/ADFD/Label/label.npy')  # Shape: (88, 2)

# Lists to hold data sets for train, val, and test
list_train = [[], []]
list_val = [[], []]
list_test = [[], []]

for id in range(1, 66):  # Only process IDs 1 to 65
    file_path = f"data/ADFD/Feature/feature_{id:02}.npy"
    data_array = np.load(file_path)  # Load the patient data

    print(data_array.shape)
    label = labels[(id-1), 0]

    if id in train_ids:
        print(f'train with label: {label}')
        list_train[0].append(data_array)
        for j in range(data_array.shape[0]):
            list_train[1].append(label)
    elif id in val_ids:
        print(f'val with label: {label}')
        list_val[0].append(data_array)
        for j in range(data_array.shape[0]):
            list_val[1].append(label)
    elif id in test_ids:
        print(f'test with label: {label}')
        list_test[0].append(data_array)
        for j in range(data_array.shape[0]):
            list_test[1].append(label)
    else:
        raise Exception(f"Something is going wrong: unexpected id {id}.")

X_train = np.concatenate(list_train[0], axis=0)
y_train = np.array(list_train[1])
X_train, y_train = shuffle(X_train, y_train, random_state=42)

X_val = np.concatenate(list_val[0], axis=0)
y_val = np.array(list_val[1])
X_val, y_val = shuffle(X_val, y_val, random_state=42)

X_test = np.concatenate(list_test[0], axis=0)
y_test = np.array(list_test[1])
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# Print shapes
print("----------------------------------------------")
print(f"Concatenated shape for X_train: {X_train.shape}")
print(f"Concatenated shape for y_train: {y_train.shape}")
print("----------------------")
np_train = (X_train, y_train)
print(f"np_train[0] shape: {np_train[0].shape}")
print(f"np_train[1] shape: {np_train[1].shape}")
print("----------------------------------------------")

print("----------------------------------------------")
print(f"Concatenated shape for X_val: {X_val.shape}")
print(f"Concatenated shape for y_val: {y_val.shape}")
print("----------------------")
np_val = (X_val, y_val)
print(f"np_val[0] shape: {np_val[0].shape}")
print(f"np_val[1] shape: {np_val[1].shape}")
print("----------------------------------------------")

print("----------------------------------------------")
print(f"Concatenated shape for X_test: {X_test.shape}")
print(f"Concatenated shape for y_test: {y_test.shape}")
print("----------------------")
np_test = (X_test, y_test)
print(f"np_test[0] shape: {np_test[0].shape}")
print(f"np_test[1] shape: {np_test[1].shape}")
print("----------------------------------------------")

(598, 256, 19)
train with label: 2
(792, 256, 19)
train with label: 2
(306, 256, 19)
train with label: 2
(706, 256, 19)
train with label: 2
(804, 256, 19)
train with label: 2
(632, 256, 19)
train with label: 2
(766, 256, 19)
train with label: 2
(792, 256, 19)
train with label: 2
(612, 256, 19)
train with label: 2
(1280, 256, 19)
train with label: 2
(768, 256, 19)
train with label: 2
(882, 256, 19)
train with label: 2
(838, 256, 19)
train with label: 2
(934, 256, 19)
train with label: 2
(900, 256, 19)
train with label: 2
(972, 256, 19)
train with label: 2
(842, 256, 19)
train with label: 2
(844, 256, 19)
train with label: 2
(918, 256, 19)
train with label: 2
(868, 256, 19)
train with label: 2
(920, 256, 19)
train with label: 2
(820, 256, 19)
val with label: 2
(834, 256, 19)
val with label: 2
(758, 256, 19)
val with label: 2
(684, 256, 19)
val with label: 2
(898, 256, 19)
val with label: 2
(826, 256, 19)
val with label: 2
(818, 256, 19)
val with label: 2
(736, 256, 19)
test with label: 2

## Importing to Binary Classification

In [ ]:
base_path = r'data\processed'

In [129]:
np.save(os.path.join(base_path, 'X_train_bin.npy'), X_train)
np.save(os.path.join(base_path, 'y_train_bin.npy'), y_train)
np.save(os.path.join(base_path, 'X_val_bin.npy'), X_val)
np.save(os.path.join(base_path, 'y_val_bin.npy'), y_val)
np.save(os.path.join(base_path, 'X_test_bin.npy'), X_test)
np.save(os.path.join(base_path, 'y_test_bin.npy'), y_test)